In [ ]:
from gff_analyser import gffClasses, gffBuilder
import hdf5plugin
from pybedtools import BedTool
import os
import sys
import pandas as pd

In [ ]:
GTF_PATH = '/home/stud3/notebooks/homo_sapiens.104.mainChr.gtf'
gtf_gene= '/home/stud3/notebooks/out/homo_sapiens.104.mainChr.gene.gtf'
gtf_exon = '/home/stud3/notebooks/out/homo_sapiens.104.mainChr.exon.gtf'
five = '/home/stud3/notebooks/out/homo_sapiens.104.mainChr.five_prime_utr.gtf'
three = '/home/stud3/notebooks/out/homo_sapiens.104.mainChr.three_prime_utr.gtf'
frag = '/home/stud3/notebooks/stomach_SM-JF1O3_rep1_fragments_sorted.bed'


In [ ]:
object_list = gffBuilder.build_gff3_class(file=GTF_PATH)

for element in object_list:
    features = element.count_features()

    element.generate_feature_gtf(gffdata_list=object_list, feature_keys=features)
    element.generate_promotor_gtf(gffdata_list=object_list)
    element.generate_tss_gtf(gffdata_list=object_list)

In [ ]:
# Getting the gene body from gene
bedtools = os.path.join('/'.join(sys.executable.split('/')[:-1]),'bedtools')
intersect_cmd = f'{bedtools} subtract -a {gtf_gene} -b {five} {three}  > out/homo_sapiens.104.mainChr.gene.gene_bodies.gtf'
os.system(intersect_cmd)

In [ ]:
# Getting the gene body from exon
bedtools = os.path.join('/'.join(sys.executable.split('/')[:-1]),'bedtools')
intersect_cmd = f'{bedtools} subtract -a {gtf_exon} -b {five} {three}  > out/homo_sapiens.104.mainChr.exon_gene_bodies.gtf'
os.system(intersect_cmd)

In [ ]:
# Getting the enhancer from gtf reference
bedtools = os.path.join('/'.join(sys.executable.split('/')[:-1]),'bedtools')
intersect_cmd = f'{bedtools} intersect -a {gtf_gene} -b stomach_SM.enhancer.bed -wa -wb > out/homo_sapiens.104.mainChr.enhancer.gtf'
os.system(intersect_cmd)

In [ ]:
bed_df = pd.read_table(frag, header=None, names=["chrom", "start", "end", "feature", "count", "strand"])

# Filter the dataframe based on a count threshold
threshold = 10
filtered_df = bed_df[bed_df["count"] > threshold]


In [ ]:
print(filtered_df)

In [ ]:
def filter_peaks_from_bed(frag_file=frag ,thres=10):

    # Load the bed file into a pandas dataframe
    bed_df = pd.read_table(frag_file, header=None, names=["chrom", "start", "end", "feature", "count", "strand"])

    # Filter the dataframe based on a count threshold
    threshold = thres
    filtered_df = bed_df[bed_df["count"] > threshold]
    
    filename = f"{frag}.peak{thres}.bed"
    # Print the filtered dataframe
    filtered_df.to_csv(filename ,sep='\t', index=None)
    return thres


In [ ]:
tresh = filter_peaks_from_bed()

In [ ]:
peakfile = 'stomach_SM-JF1O3_rep1_fragments_sorted.bed.peak10.bed'

In [ ]:
# Getting the gene body from exon
bedtools = os.path.join('/'.join(sys.executable.split('/')[:-1]),'bedtools')
intersect_cmd = f'{bedtools} intersect -a {GTF_PATH} -b {peakfile} -wa -wb > out/homo_sapiens.104.mainChr.peak{tresh}.gtf'
os.system(intersect_cmd)